# code to save brain data into a .mat file

In [1]:
import os
import glob

from scipy.io import savemat

from nilearn.image import load_img
import numpy as np
import h5py
import pandas as pd
import npp

#get list of all files
#loop through first 3 and concantenate them

#open brain file
# brainpath='../data/HBN/clean/sub-NDARAA075AMK/ses-HBNsiteSI/sub-NDARAA075AMK_ses-HBNsiteSI_task-rest_acq-VARIANTObliquity_space-fsLR_den-91k_bold_clean_smooth2.dtseries.nii'
# brainimg = load_img(brainpath) # load func img
# func_data=np.array(brainimg.dataobj)#add file to list
# #save list

In [2]:
dx_final=pd.read_csv('../data/phenotype/dx_onehot.csv')

In [4]:
dx_final

,Identifiers,Age,Sex,Site,ADHD-Combined Type,ADHD-Hyperactive/Impulsive Type,ADHD-Inattentive Type,Acute Stress Disorder,Adjustment Disorders,Agoraphobia,...,Trichotillomania (Hair-Pulling Disorder),Unspecified Anxiety Disorder,Unspecified Attention-Deficit/Hyperactivity Disorder,Unspecified Bipolar and Related Disorder,Unspecified Depressive Disorder,Unspecified Feeding or Eating Disorder,Unspecified Intellectual Disability,Unspecified Neurodevelopmental Disorder,Unspecified Tic Disorder,Unspecified Trauma- and Stressor-Related Disorder
0,NDARAA075AMK,6.728040,female,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NDARAA112DMH,5.545744,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NDARAA117NEJ,7.475929,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NDARAA306NT2,21.216746,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NDARAA358BPN,11.853296,male,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,NDARZT957CWG,16.034565,male,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4763,NDARZU401RCU,17.663586,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4764,NDARZV458LDP,15.763175,female,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4765,NDARZX745YLZ,16.288501,male,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## save RU ASD and NDG

In [5]:
dx_final=pd.read_csv('../data/phenotype/dx_onehot.csv') # load demographics
output_path='../data/for_matlab/'
site='ses-HBNsiteRU'
#task='task-rest_run-2'
#task='task-movieTP'
task='task-movieDM'

for task in ['task-rest_run-1','task-rest_run-2','task-movieTP','task-movieDM']:
    subject_list=[]
    dx_list=[]
    age_list=[]
    sex_list=[]
    brain_data_files=[]

    brain_data=[]

    dxs=[('ndg','No Diagnosis Given'),('asd','Autism Spectrum Disorder')]
    for dx,dx_name in dxs:
        print(dx, dx_name)
        for index, row in dx_final[dx_final[dx_name] == 1].iterrows():
            s=row['Identifiers']
            age=row['Age']
            sex=row['Sex']
            dir_path = f'../data/HBN/clean/sub-{s}/{site}/sub-{s}_{site}_{task}*clean_smooth2.dtseries.nii'
            try:
                brainfile=glob.glob(f"{dir_path}")[0]
                brainimg = load_img(brainfile) # load func img
                func_data=np.array(brainimg.dataobj)[10:,:]#remove first 10 timepoints, add file to list
                func_data=npp.rescale( npp.demean(func_data.T) ).T
                brain_data.append(func_data)
                brain_data_files.append(brainfile)
                subject_list.append(f'sub-{s}')
                dx_list.append(dx)
                age_list.append(age)
                sex_list.append(sex)
            except:
                pass
                #print('no or bad file')

    #we want brain x time x subject
    brain_data_array = np.transpose(   np.stack( brain_data )  )
    brain_data_array.shape

    #save h5
    file_path = f'{output_path}all_{site}_{task}.h5'
    with h5py.File(file_path, 'w') as f:
        f.create_dataset('data', data=brain_data_array)



    #export metadata to csv
    data = {'subject': subject_list, 'dx': dx_list, 'age': age_list, 'sex':sex_list, 'path': brain_data_files}
    df = pd.DataFrame(data)
    df.to_csv(f'{output_path}all_{site}_{task}.csv', index=False)

ndg No Diagnosis Given
asd Autism Spectrum Disorder
ndg No Diagnosis Given
asd Autism Spectrum Disorder
ndg No Diagnosis Given
asd Autism Spectrum Disorder
ndg No Diagnosis Given
asd Autism Spectrum Disorder


In [12]:
len(glob.glob('../data/HBN/clean/sub-*/ses-HBNsiteRU/'))

658

In [13]:
len(glob.glob('../data/HBN/clean/sub-*/ses-HBNsiteCBIC/'))

708

In [14]:
len(glob.glob('../data/HBN/clean/sub-*/ses-HBNsiteSI/'))

222

In [17]:
len(glob.glob('../data/HBN/clean/sub-*/ses-HBNsiteCUNY/'))

56